In [1]:
from omegaconf import DictConfig, OmegaConf
import hydra
import dataload
import embed
import evaluate
from utils.data_utils import data_to_kg, extract_ents
import preprocess
from sklearn.metrics import accuracy_score, confusion_matrix
import fire
import sys
import tqdm
import pandas as pd
import numpy as np
import kgbench as kg
from collections import Counter
import matplotlib.pyplot as pl
import random as rd
from operator import itemgetter
import torch
import torch.nn.functional as F
from utils import RDF_NUMBER_TYPES, get_relevant_relations, add_triple, get_p_types
from kgbench.load import Data
from typing import List, Sequence, Tuple

from kgbench import load, tic, toc, d
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
from utils import URI_PREFIX
import numpy as np
from sklearn.neighbors import LocalOutlierFactor






c:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = kg.load(name= "mdgenre", prune_dist = None)

In [50]:
len(data.i2e)

349344

In [47]:
data.e2i

{(0,
  ('"Crocodile" Dundee',
   'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (1,
  ('#Selfie',
   'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (2, ('&ME', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (3, ("''C.B.''", 'http://www.w3.org/2001/XMLSchema#string')): 349343,
 (4, ("'71", 'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (5,
  ("'A' gai wak",
   'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (6, ("'Tex' O'Brien", 'http://www.w3.org/2001/XMLSchema#string')): 349343,
 (7,
  ("'Twas the Night",
   'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (8,
  ('(500) Days of Summer',
   'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (9, ('(Space Invader)', 'http://www.w3.org/2001/XMLSchema#string')): 349343,
 (10, ('***', 'http://www.w3.org/2001/XMLSchema#string')): 349343,
 (11,
  ('-0070-04-01T00:00:00Z',
   'http://www.w3.org/2001/XMLSch

In [7]:
relevent_relations = get_relevant_relations(
    get_p_types(data), relevant_types=RDF_NUMBER_TYPES,r2i=data.r2i)

In [9]:
def encode_number_sublist(df: torch.Tensor, i2e: List[str]) -> torch.Tensor:
    sub_df = df.clone()
    for i in range(len(sub_df)):
        sub_df[i, 1] = torch.tensor(
            float(i2e[sub_df[i, 2]][0]), dtype=torch.float32)
    sub_df = sub_df[:, :2]
    return sub_df


In [15]:
data.i2r[relevent_relations[1] ]

'http://www.wikidata.org/prop/direct/P2047'

In [20]:
sub_df

tensor([[259177,      7],
        [192144,    135],
        [113554,     94],
        ...,
        [101913,    105],
        [136621,    100],
        [204579,    104]], dtype=torch.int32)

In [18]:
data.i2r[relevent_relations[1]] 

False

In [23]:
data.e2i

{(0,
  ('"Crocodile" Dundee',
   'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (1,
  ('#Selfie',
   'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (2, ('&ME', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (3, ("''C.B.''", 'http://www.w3.org/2001/XMLSchema#string')): 349343,
 (4, ("'71", 'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (5,
  ("'A' gai wak",
   'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (6, ("'Tex' O'Brien", 'http://www.w3.org/2001/XMLSchema#string')): 349343,
 (7,
  ("'Twas the Night",
   'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (8,
  ('(500) Days of Summer',
   'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (9, ('(Space Invader)', 'http://www.w3.org/2001/XMLSchema#string')): 349343,
 (10, ('***', 'http://www.w3.org/2001/XMLSchema#string')): 349343,
 (11,
  ('-0070-04-01T00:00:00Z',
   'http://www.w3.org/2001/XMLSch

In [26]:
data.e2i

{(0,
  ('"Crocodile" Dundee',
   'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (1,
  ('#Selfie',
   'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (2, ('&ME', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (3, ("''C.B.''", 'http://www.w3.org/2001/XMLSchema#string')): 349343,
 (4, ("'71", 'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (5,
  ("'A' gai wak",
   'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (6, ("'Tex' O'Brien", 'http://www.w3.org/2001/XMLSchema#string')): 349343,
 (7,
  ("'Twas the Night",
   'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (8,
  ('(500) Days of Summer',
   'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString')): 349343,
 (9, ('(Space Invader)', 'http://www.w3.org/2001/XMLSchema#string')): 349343,
 (10, ('***', 'http://www.w3.org/2001/XMLSchema#string')): 349343,
 (11,
  ('-0070-04-01T00:00:00Z',
   'http://www.w3.org/2001/XMLSch

In [25]:
for d in sub_df:
    if data.i2e[d[0]] not in data.e2i:
        print(data.e2i[data.i2e[d[0]]])
        print("why")
        break
#n data.i2e:

KeyError: ('http://www.wikidata.org/entity/Q82538731', 'iri')

In [12]:
rr = relevent_relations[1] 
df = data.triples.clone()
df = df[df[:, 1]== rr]
sub_df = encode_number_sublist(df, data.i2e)

In [13]:
sub_df

tensor([[259177,      7],
        [192144,    135],
        [113554,     94],
        ...,
        [101913,    105],
        [136621,    100],
        [204579,    104]], dtype=torch.int32)

In [4]:
data.i2r

['http://www.wikidata.org/prop/direct/P10',
 'http://www.wikidata.org/prop/direct/P102',
 'http://www.wikidata.org/prop/direct/P103',
 'http://www.wikidata.org/prop/direct/P1038',
 'http://www.wikidata.org/prop/direct/P1040',
 'http://www.wikidata.org/prop/direct/P1050',
 'http://www.wikidata.org/prop/direct/P106',
 'http://www.wikidata.org/prop/direct/P1066',
 'http://www.wikidata.org/prop/direct/P108',
 'http://www.wikidata.org/prop/direct/P109',
 'http://www.wikidata.org/prop/direct/P118',
 'http://www.wikidata.org/prop/direct/P119',
 'http://www.wikidata.org/prop/direct/P1191',
 'http://www.wikidata.org/prop/direct/P1196',
 'http://www.wikidata.org/prop/direct/P1283',
 'http://www.wikidata.org/prop/direct/P1303',
 'http://www.wikidata.org/prop/direct/P1317',
 'http://www.wikidata.org/prop/direct/P1327',
 'http://www.wikidata.org/prop/direct/P1340',
 'http://www.wikidata.org/prop/direct/P1343',
 'http://www.wikidata.org/prop/direct/P1344',
 'http://www.wikidata.org/prop/direct/P135'

In [6]:
data = preprocess.bin_numbers2(data=data)

('http://www.wikidata.org/entity/Q3595806', 'iri')
('http://www.wikidata.org/entity/Q455043', 'iri')
('http://www.wikidata.org/entity/Q1631794', 'iri')
('http://www.wikidata.org/entity/Q57604', 'iri')
('http://www.wikidata.org/entity/Q18361225', 'iri')
('http://www.wikidata.org/entity/Q316032', 'iri')
('http://www.wikidata.org/entity/Q26053', 'iri')
('http://www.wikidata.org/entity/Q382393', 'iri')
('http://www.wikidata.org/entity/Q121507', 'iri')
('http://www.wikidata.org/entity/Q237925', 'iri')
('http://www.wikidata.org/entity/Q934270', 'iri')
('http://www.wikidata.org/entity/Q957711', 'iri')
('http://www.wikidata.org/entity/Q443361', 'iri')
('http://www.wikidata.org/entity/Q1751820', 'iri')
('http://www.wikidata.org/entity/Q435415', 'iri')
('http://www.wikidata.org/entity/Q2707974', 'iri')
('http://www.wikidata.org/entity/Q353978', 'iri')
('http://www.wikidata.org/entity/Q553575', 'iri')
('http://www.wikidata.org/entity/Q1349729', 'iri')
('http://www.wikidata.org/entity/Q2977664', '